<a href="https://colab.research.google.com/github/fsilva0703/AxpoTrade/blob/main/project01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
### **Project: Apply Lightweight Fine-Tuning to a Foundation Model**

#### **1. Project Setup**

# Install required libraries with compatible versions
!pip install transformers datasets evaluate peft

#### **2. Dataset Loading and Preprocessing**
from datasets import load_dataset
from transformers import AutoTokenizer

# Load Yelp Polarity dataset and take a small subset
dataset = load_dataset("yelp_polarity")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(500))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(100))

# Load the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

#### **3. Foundation Model Loading and Initial Evaluation**
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np

# Load the pretrained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Set training arguments for evaluation
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    report_to="none"
)

# Evaluate the foundation model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Evaluating the original model...")
original_results = trainer.evaluate()
print("Original Model Accuracy:", original_results["eval_accuracy"])

#### **4. Parameter-Efficient Fine-Tuning (PEFT) Setup**
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Apply PEFT to the foundation model
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

#### **5. Training the PEFT Model**

# Update the Trainer to use the PEFT model
trainer.model = peft_model

print("Training the PEFT model...")
trainer.train()

# Save the fine-tuned model
peft_model.save_pretrained("./peft-bert-yelp")


#### **6. Loading and Evaluating the Fine-Tuned Model**

from peft import AutoPeftModelForSequenceClassification

# Load the fine-tuned PEFT model
print("Loading the fine-tuned model...")
fine_tuned_model = AutoPeftModelForSequenceClassification.from_pretrained("./peft-bert-yelp")

# Update the Trainer to use the fine-tuned model
trainer.model = fine_tuned_model

print("Evaluating the fine-tuned model...")
fine_tuned_results = trainer.evaluate()
print("Fine-Tuned Model Accuracy:", fine_tuned_results["eval_accuracy"])


Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Original Model Accuracy: 0.55
trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700
Training the PEFT model...


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,No log,0.674981,0.006100,0.570000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading the fine-tuned model...
Evaluating the fine-tuned model...


Fine-Tuned Model Accuracy: 0.57
